In [1]:
import sys
import pymysql

from PyQt5 import QtWidgets
from PyQt5.QtWidgets import QApplication, QMainWindow, QDialog, QMessageBox, QTableWidgetItem, QTextBrowser
from PyQt5.uic import loadUi

class QueryWindow(QDialog): 
    def __init__(self, main_window, connection):
        super().__init__()
        loadUi("query_window.ui", self)
        self.main_window = main_window
        self.connection = connection

        # Combo/Desplegable

        self.query_combo.addItem('Visualizar Tabla')
        self.query_combo.addItem('Agregar Estadio')

        # Boton Volver
        self.query_volver.clicked.connect(self.return_to_main)


        self.query_query.clicked.connect(self.execute_query) # Query visualiacion


        self.query_insert.clicked.connect(self.insert_data) # Query Agregar

    def insert_data(self):
        nombre = self.query_nombre.text()
        direccion = self.query_direccion.text()
        comuna = self.query_comuna.text()
        aforo = self.query_aforo.text()

        if not (nombre and direccion and comuna and aforo):
            QMessageBox.critical(self, 'Error', 'Llena la informacion!!!')
            return 
        insert_query = "INSERT INTO estadio (nombre, direccion, comuna, aforo) VALUES (%s, %s, %s, %s)"

        try:
            with self.connection.cursor() as cursor:
                cursor.execute(insert_query, (nombre, direccion, comuna, aforo))
                self.connection.commit()
                QMessageBox.information(self, 'Success', 'Data inserted')
                self.execute_query()
        except pymysql.MySQLError as e:
            QMessageBox.critical(self, 'Insertion Error', str(e))




    def execute_query(self):
        if not self.connection:
            QMessageBox.critical(self, "Error", "No database connection")
            return

        query = self.get_query(self.query_combo.currentText())
        try:
            with self.connection.cursor() as cursor:
                cursor.execute(query)
                result = cursor.fetchall()
                print(result)
                self.display_results(result)
        except pymysql.MySQLError as e:
            QMessageBox.critical(self, "Query Error", str(e))

    def get_query(self, query_name):
        tabla = self.query_table.text()
        queries = {
            "Visualizar Tabla": f"SELECT * FROM {tabla}", 
        
        }
        return queries.get(query_name, "")
    

    def display_results(self, results):
        print(results)
        self.mostrar.setRowCount(len(results))
        self.mostrar.setColumnCount(len(results[0]) if results else 0)
        for i, row in enumerate(results):
            for j, item in enumerate(row):
                self.mostrar.setItem(i, j, QtWidgets.QTableWidgetItem(str(item)))   
    



    def return_to_main(self):
        self.main_window.show()
        self.close()

class MainWindow(QMainWindow):
    
    def __init__(self):
        super().__init__()
        loadUi("main_window.ui", self)
        self.connection = None

        # Conectar a SQL
        
        self.main_connect.clicked.connect(self.connect_to_database)

    def connect_to_database(self):
        # Obtener los datos de conexión
        host = self.main_txt_host.text()
        user = self.main_txt_user.text()
        password = self.main_txt_pass.text()

        # Conectar con WorkBench
        
        try:
            self.connection = pymysql.connect(
                host=host,
                user=user,
                password=password,
                database='anfp'  # Especificar la base de datos aquí
            
            )
            QMessageBox.information(self, "Success", "Connected to database!")

            # Abrir la ventana de consulta
            self.query_window = QueryWindow(self, self.connection)
            self.query_window.show()
            self.hide()  # Ocultar la ventana principal
        except pymysql.MySQLError as e:
            QMessageBox.critical(self, "Connection Error", str(e))



if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_window = MainWindow()
    main_window.show()
    sys.exit(app.exec_())

C:\Users\chave\AppData\Local\Temp\ipykernel_17736\490646049.py:8: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  class QueryWindow(QDialog):
C:\Users\chave\AppData\Local\Temp\ipykernel_17736\490646049.py:91: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  class MainWindow(QMainWindow):


(('a', 'a', 'a', 1), ('El Teniente', 'Los Huasos', 'Rancagua', 14500), ('Estadio ', 'Direccion', 'Comuna', 69), ('Estadio nuevo', 'Direccion nueva', 'Comuna Nueva', 42000), ('La Florida', 'Vicuna Mackenna', 'La Florida', 12000), ('Monumental', 'Pedreros', 'Macul', 47017), ('Municipal de Calama', 'Los Cobres', 'Calama', 12000), ('Nacional', 'Grecia', 'Nunoa', 50000), ('Nelson Oyarzun', 'Pleno Centro', 'Chillan', 12000), ('San Carlos', 'Apoquindo', 'Las Condes', 25000), ('Sausalito', 'Laguna Sausalito', 'Vina del Mar', 25000))
(('a', 'a', 'a', 1), ('El Teniente', 'Los Huasos', 'Rancagua', 14500), ('Estadio ', 'Direccion', 'Comuna', 69), ('Estadio nuevo', 'Direccion nueva', 'Comuna Nueva', 42000), ('La Florida', 'Vicuna Mackenna', 'La Florida', 12000), ('Monumental', 'Pedreros', 'Macul', 47017), ('Municipal de Calama', 'Los Cobres', 'Calama', 12000), ('Nacional', 'Grecia', 'Nunoa', 50000), ('Nelson Oyarzun', 'Pleno Centro', 'Chillan', 12000), ('San Carlos', 'Apoquindo', 'Las Condes', 2500